In [1]:
import re
import os
import numpy as np
from statistics import mode
import pickle
import datetime as dt
import scipy.spatial

In [2]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet,stopwords
import string
import spacy

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
from airtable import Airtable
from IPython.core.display import display, HTML

In [4]:
lemmatizer = WordNetLemmatizer() 

In [5]:
nlp = spacy.load('en', disable=['parser', 'ner'])
stop_words = set(stopwords.words('english'))

In [6]:
sam =nlp('badly')

" ".join([token.lemma_ for token in sam])

# clean('i am going to be worked up')

'badly'

In [7]:

def remove_stopwords(tokenized_text): 
    text = tokenized_text.split(' ')
    text = " ".join([word for word in text if word not in stop_words])
    return text
 
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def clean(aa):
    aa = aa.lower()
    aa = remove_stopwords(aa)
    aa = re.sub('[%s]' % re.escape('/-()'), ' ', aa)
    table = str.maketrans('', '', string.punctuation)
    aa = aa.translate(table)
    aa= re.sub(r"\s+", ' ', aa)
    aa = aa.strip()
    # for lemmatization (going=go, boys=boy)
    aa = nlp(aa)
    aa = " ".join([token.lemma_ for token in aa])
    
    aa = re.sub(r'\b(\w+)( \1\b)+', r'\1', aa)
    return aa

## Connection to Airtable
- Ensure no filters are applied before running the connect code

In [9]:
base_key ='appeOkjFAXjcPBnTJ'
key_api = '*****************'
table_name ='Imported table Edited'

In [11]:
airtable = Airtable(base_key, table_name, api_key=key_api)
print(airtable)

<Airtable table:Imported table Edited>


In [12]:
# records = airtable.get_all(maxRecords=2)
records = airtable.get_all(view='database',fields=['Capabilties','Domain'])
df = pd.DataFrame.from_records((r['fields'] for r in records))

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1084 entries, 0 to 1083
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Capabilties  1084 non-null   object
 1   Domain       1084 non-null   object
dtypes: object(2)
memory usage: 17.1+ KB


### Domain was in list format

In [14]:
pd.set_option('display.max_colwidth', None)
df.head(7)

,Capabilties,Domain
0,Ability to review outstanding Service Requests,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
1,Ability to view a GIS map,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
2,Ability to update close (change Status) a Service Requests,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
3,Ability to view the location of a permit on a map,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
4,Ability to view outstanding Service Requests,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
5,Ability to change the owner on a Service Requests,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
6,Ability to view the location of a license on a map,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"


- Capabilities without a specific domain

Numbers of Capabilties and Domain should be equal

In [15]:
df[df['Domain'].isnull()]

,Capabilties,Domain


## Exploding the domain column 

In [16]:
df["Domain"].apply(pd.Series)\
    .merge(df, right_index = True, left_index = True)\
    .head(5)

,0,1,2,3,4,5,6,7,Capabilties,Domain
0,Permits,Licenses,Citizen Request,Planning and Zoning,Inspections,Code Enforcement,Finance,NaN,Ability to review outstanding Service Requests,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
1,Permits,Licenses,Citizen Request,Planning and Zoning,Inspections,Code Enforcement,Finance,NaN,Ability to view a GIS map,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
2,Permits,Licenses,Citizen Request,Planning and Zoning,Inspections,Code Enforcement,Finance,NaN,Ability to update close (change Status) a Service Requests,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
3,Permits,Licenses,Citizen Request,Planning and Zoning,Inspections,Code Enforcement,Finance,NaN,Ability to view the location of a permit on a map,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"
4,Permits,Licenses,Citizen Request,Planning and Zoning,Inspections,Code Enforcement,Finance,NaN,Ability to view outstanding Service Requests,"[Permits, Licenses, Citizen Request, Planning and Zoning, Inspections, Code Enforcement, Finance]"


In [17]:
df_melt =df["Domain"].apply(pd.Series)\
    .merge(df, right_index = True, left_index = True)\
    .drop(["Domain"], axis = 1) \
    .melt(id_vars = ['Capabilties'], value_name = "Domain")\
    .drop("variable", axis = 1)\
    .dropna(subset=['Domain'])\
    .reset_index(drop=True)

In [18]:
df_melt[df_melt.Domain=='Finance'].reset_index(drop=True).to_csv('financials_capabilities.csv')

In [19]:
df_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4357 entries, 0 to 4356
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Capabilties  4357 non-null   object
 1   Domain       4357 non-null   object
dtypes: object(2)
memory usage: 68.2+ KB


In [20]:
#remove all duplicate capabilities
df_AllCapabilities = df_melt.drop_duplicates(subset=['Capabilties'])

In [21]:
df_AllCapabilities.head()

,Capabilties,Domain
0,Ability to review outstanding Service Requests,Permits
1,Ability to view a GIS map,Permits
2,Ability to update close (change Status) a Service Requests,Permits
3,Ability to view the location of a permit on a map,Permits
4,Ability to view outstanding Service Requests,Permits


## Now the domains can be filtered

In [22]:
# df_melt.groupby(by=['Capabilties','Domain']).count()\
#                                             .reset_index()
df_melt['Domain'].value_counts()

Finance                708
Inspections            679
Permits                613
Citizen Request        572
Planning and Zoning    571
Licenses               562
Code Enforcement       537
Portal                 115
Name: Domain, dtype: int64

In [23]:
def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

In [24]:
# pd.set_option('display.max_colwidth', None)
display_side_by_side([df_melt[df_melt["Domain"]=='Permits'].head(5)
                      , df_melt[df_melt["Domain"]=='Inspections'].head(5)], ['Permits', 'Inspections'])


,Capabilties,Domain
0,Ability to review outstanding Service Requests,Permits
1,Ability to view a GIS map,Permits
2,Ability to update close (change Status) a Service Requests,Permits
3,Ability to view the location of a permit on a map,Permits
4,Ability to view outstanding Service Requests,Permits
,Capabilties,Domain
169,Ability to add an address to a Complaint,Inspections
174,Ability to add an address to Equipment,Inspections
175,Ability to add an address to an Inspection,Inspections
205,Ability to allow permit holders to request inspections,Inspections


In [25]:
domain_list = df_melt['Domain'].value_counts().keys().tolist()
Unwanted = ['Utilities','Portal','Salesforce']
domain_list = [e for e in domain_list if e not in Unwanted]

In [26]:
print(len(domain_list))
domain_list

7


['Finance',
 'Inspections',
 'Permits',
 'Citizen Request',
 'Planning and Zoning',
 'Licenses',
 'Code Enforcement']

In [27]:
# create new database for each domain and save it in a dict
dict_of_domainsdf = {}
for i in domain_list:
    dict_of_domainsdf["df_{}".format(i)] = df_melt[df_melt["Domain"]== i].reset_index(drop=True)

In [28]:
print(dict_of_domainsdf.keys())
dict_of_domainsdf['df_Finance'].head()

dict_keys(['df_Finance', 'df_Inspections', 'df_Permits', 'df_Citizen Request', 'df_Planning and Zoning', 'df_Licenses', 'df_Code Enforcement'])


,Capabilties,Domain
0,Ability to create fee overrides,Finance
1,Ability to generate an invoice for inspections,Finance
2,Ability to assign fees to inspection,Finance
3,Ability to manage Surety Bonds,Finance
4,Ability to integrate our cart with 3rd party gateways to handle specific payment methods,Finance


In [29]:
df_Finance = dict_of_domainsdf['df_Finance']
df_Inspections = dict_of_domainsdf['df_Inspections']
df_Permits = dict_of_domainsdf['df_Permits']
df_Citizen = dict_of_domainsdf['df_Citizen Request']
df_Planning = dict_of_domainsdf['df_Planning and Zoning']
df_Licenses = dict_of_domainsdf['df_Licenses']
df_Code = dict_of_domainsdf['df_Code Enforcement']

In [30]:
df_Code.head(5)

,Capabilties,Domain
0,Ability to search violation codes,Code Enforcement
1,"Ability to view a dashboard with all new and open complaints, and recent activities",Code Enforcement
2,Ability to reopen a closed complaint,Code Enforcement
3,Ability to create and track payment plans for monetary sanctions,Code Enforcement
4,Ability to track actions that need to be taken against a code violation,Code Enforcement


### Cleaning each domain text

In [31]:
df_name = [df_Finance, df_Inspections, df_Permits,df_Citizen,df_Planning, df_Licenses, df_Code]
for i in (df_Finance, df_Inspections, df_Permits,df_Citizen,df_Planning, df_Licenses, df_Code,df_AllCapabilities):
    i.drop(columns=['Domain'],inplace=True)
    i['clean_text'] = i['Capabilties'].apply(lambda x:clean(x))

/Users/temitopebadekale/opt/anaconda3/envs/clariti/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/temitopebadekale/opt/anaconda3/envs/clariti/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [32]:
df_AllCapabilities.head(5)

,Capabilties,clean_text
0,Ability to review outstanding Service Requests,ability review outstanding service request
1,Ability to view a GIS map,ability view gis map
2,Ability to update close (change Status) a Service Requests,ability update close change status service request
3,Ability to view the location of a permit on a map,ability view location permit map
4,Ability to view outstanding Service Requests,ability view outstanding service request


### create csv file for each domain
- run if updating connection to airtable

### create a new folder in app2_file folder with the current time

In [33]:
mydir = os.path.join(os.getcwd(),'app2_file', dt.datetime.now().strftime('%Y-%b-%d_%H:%M:%S'))
os.makedirs(mydir)

In [34]:
for i,k in zip((df_AllCapabilities,df_Finance, df_Inspections, df_Permits,df_Citizen,df_Planning, df_Licenses, df_Code),
             ('All_Capabilities','Finance_domain', 'Inspections_domain', 'Permits_domain',
              'Citizen_domain','Planning_domain','Licenses_domain','Code_domain')):
    i.to_csv(f'{mydir}/{k}.csv',index = False)

In [35]:
pd.read_csv(f'{mydir}/Citizen_domain.csv').head()

,Capabilties,clean_text
0,Ability to call sets of code (classes) in a specific order of execution (Apex Trigger Framework),ability call set code class specific order execution apex trigger framework
1,Ability to lookup objects,ability lookup object
2,Ability to recover deleted Records (some considerations apply),ability recover delete record some consideration apply
3,Ability for emails sent to citizens to contain attachments,ability email send citizen contain attachment
4,Ability to provide data to DOL that is compatible with SQL Server 2016.,ability provide datum dol compatible sql server 2016


In [36]:
pd.read_csv(f'{mydir}/All_Capabilities.csv').head()

,Capabilties,clean_text
0,Ability to review outstanding Service Requests,ability review outstanding service request
1,Ability to view a GIS map,ability view gis map
2,Ability to update close (change Status) a Service Requests,ability update close change status service request
3,Ability to view the location of a permit on a map,ability view location permit map
4,Ability to view outstanding Service Requests,ability view outstanding service request


In [37]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
module_url = './tfhub_modules/063d866c06683311b44b4992fd46003be952409c'

# Import the Universal Sentence Encoder's TF Hub module
model = hub.load(module_url)

### Train the model on each domain files
- run if updating connection to airtable

In [38]:
raw_text = []
corpus = []
for i in (df_AllCapabilities,df_Finance, df_Inspections, df_Permits,df_Citizen,df_Planning, df_Licenses, df_Code):
    x = i['Capabilties'].tolist()
    raw_text.append(x)
    y = i['clean_text'].tolist()
    corpus.append(y)
    
corpus_embeddings = []
for corp in corpus:
    corpus_embeddings.append(model(corp)) 

In [39]:
print(len(corpus))
model(corpus[0])

8


<tf.Tensor: id=5314, shape=(1084, 512), dtype=float32, numpy=
array([[-0.0140986 , -0.00868113, -0.0408753 , ..., -0.0393325 ,
         0.00806392,  0.01393174],
       [ 0.05388511, -0.03091858, -0.0481126 , ..., -0.02380868,
        -0.02116611, -0.02129252],
       [ 0.00075236, -0.01626579, -0.06492362, ...,  0.04181892,
        -0.02195569,  0.04346166],
       ...,
       [ 0.03958058, -0.00426544, -0.05827956, ..., -0.04025534,
         0.05216417,  0.03156464],
       [ 0.06561497, -0.00097827,  0.0561373 , ..., -0.02502465,
         0.0126515 , -0.03551616],
       [ 0.05993295,  0.01314416, -0.01566146, ...,  0.01540744,
        -0.05160278, -0.06540434]], dtype=float32)>

### Saving corpus_embedding as a plk file to be used by other .py file
- run if updating connection to airtable
- run the dump section

In [40]:
PIK=f'{mydir}/All_Domains.plk'
with open(PIK, "wb") as f:
    pickle.dump(len(corpus_embeddings), f)
    for value in corpus_embeddings:
        pickle.dump(value, f)

## using pickled and saved files

In [58]:
all_files = os.listdir("app2_file")
all_files = filter(lambda x: x != '.DS_Store', all_files)
all_files = sorted(all_files ,key = lambda date: datetime.datetime.strptime(date, '%Y-%b-%d_%X'),reverse=True)
file_path = os.path.join(os.getcwd(),'app2_file',all_files[0])
print(all_files,'\n')
print(file_path)

['2021-Feb-12_09:05:55', '2021-Jan-19_09:49:59', '2021-Jan-18_13:47:42', '2021-Jan-18_12:34:37', '2021-Jan-18_12:21:04'] 

/Users/temitopebadekale/Documents/clariiti/app2_file/2021-Feb-12_09:05:55


In [61]:
#reads each file from the above path into pd dataframe
Alldomain_raw=[]
for x in ('All_Capabilities','Finance_domain', 'Inspections_domain', 'Permits_domain',
              'Citizen_domain','Planning_domain','Licenses_domain','Code_domain'):
    x= pd.read_csv(f'{file_path}/{x}.csv')['Capabilties'].tolist()
    Alldomain_raw.append(x)

In [62]:
#reads the trained model of each file
pickle_domain= []
Path_dir=f'{file_path}/All_Domains.plk'
with open(Path_dir, "rb") as f:
    for _ in range(pickle.load(f)):
        pickle_domain.append(pickle.load(f))

In [63]:
### Test the code
queries=['Ability to apply for a new address']
clean_queries = [clean(i)for i in queries]
query_embeddings = model(clean_queries)

In [66]:
domain_id = ['All','Finance', 'Inspections', 'Permits', 'Citizen Request',
             'Planning and Zoning', 'Licenses', 'Code Enforcement']
closest_n = 10;domain = 0;sc=60
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], pickle_domain[domain], "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    print(f'For {domain_id[domain]} domain  ','\n')
    print("\n======================\n")
    print("Query:", query,"\n")
    print(f'Top {closest_n} most similar sentences in corpus:')

    for idx, distance in results[0:closest_n]:
        x =(1-distance)*100
        if x >= sc:
            print(Alldomain_raw[domain][idx].strip(), f"Score {x:.0f}%" )

For All domain   



Query: Ability to apply for a new address 

Top 10 most similar sentences in corpus:
Ability to apply for a new address Score 100%
Ability to create new addresses Score 85%
Ability to add an address to a  license Score 72%
Ability to add an address to a permit Score 71%
Ability to add an address to an Application Score 70%
Ability to create new addresses in bulk (future) Score 68%
Ability to manage addresses Score 67%
Ability to add an address to a Complaint Score 66%
Ability to associate an address on a permit Score 64%
Ability to add an address to Equipment Score 63%
